# Setup

In [4]:
# Imports
import cv2
import copy
import itertools
import keras
import logging
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import re
import scipy.misc
import tensorflow as tf
import time

from scipy import sparse
from scipy import ndimage
from heapq import heappush, heappop
from networkx_viewer import Viewer
from sklearn.feature_extraction import image
from matplotlib import colors as mcolors
from sys import stdout

#%matplotlib inline

Using TensorFlow backend.


# Image to graph

In [5]:
def _make_edges_3d(n_x, n_y, n_z=1):
    """Returns a list of edges for a 3D image.
    Parameters
    ===========
    n_x: integer
        The size of the grid in the x direction.
    n_y: integer
        The size of the grid in the y direction.
    n_z: integer, optional
        The size of the grid in the z direction, defaults to 1
    """
    vertices = np.arange(n_x * n_y * n_z).reshape((n_x, n_y, n_z))
    edges_deep = np.vstack((vertices[:, :, :-1].ravel(),
                            vertices[:, :, 1:].ravel()))
    edges_right = np.vstack((vertices[:, :-1].ravel(),
                             vertices[:, 1:].ravel()))
    edges_down = np.vstack((vertices[:-1].ravel(), vertices[1:].ravel()))
    edges = np.hstack((edges_deep, edges_right, edges_down))
    return edges

In [6]:
def _compute_altitude_3d(edges, img):
    n_x, n_y, n_z = img.shape
    gradient = np.abs(img[edges[0] // (n_y * n_z),
                          (edges[0] % (n_y * n_z)) // n_z,
                          (edges[0] % (n_y * n_z)) % n_z] -
                          img[edges[1] // (n_y * n_z),
                          (edges[1] % (n_y * n_z)) // n_z,
                          (edges[1] % (n_y * n_z)) % n_z])
    return gradient

In [7]:
def img_to_graph(image):
    
    dtype=None
    image = np.atleast_3d(image)
    n_x, n_y, n_z = image.shape
    edges = _make_edges_3d(n_x, n_y, n_z)
    weights = _compute_altitude_3d(edges, image)
    diag = image.ravel()
    n_voxels = diag.size
    diag_idx = np.arange(n_voxels)
    i_idx = np.hstack((edges[0], edges[1]))
    j_idx = np.hstack((edges[1], edges[0]))
    matrix = sparse.coo_matrix((np.hstack((weights, weights, diag)),
                              (np.hstack((i_idx, diag_idx)),
                               np.hstack((j_idx, diag_idx)))),
                              (n_voxels, n_voxels),
                              dtype=dtype)
    graph = nx.from_scipy_sparse_matrix(matrix)
    graph.remove_edges_from(graph.selfloop_edges())
    
    mapping = map_node(image)
    values = get_altitude_map(image)
    
    graph = nx.relabel_nodes(graph, mapping)
    nx.set_node_attributes(graph,'value',values=values)
    
    return graph

In [8]:
def get_altitude_map(img):
    values = dict()
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            values[(row,col)] = img[row,col][0]
    
    return values

In [9]:
def map_node(img):
    
    assert isinstance(img, np.ndarray), "Not an image"
    
    mapping = dict()
    
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            mapping[row * img.shape[1] + col] = ((row), (col))

    return mapping

In [10]:
def get_image_positions(img):
    positions = dict()
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            positions[(row,col)] = (col,row)
    return positions

In [11]:
def view_graph(img, graph, pixel_values=False, 
               figurename='graph.jpg'):
    node_size = 1000
    node_color = 'b'
    seed_color = 'r'
    colors = ["pink","orange","brown","red","green",
              "orange","beige","turquoise","blue"]
    
    plt.figure(figsize=(img.shape[0],img.shape[1]))
    
    positions = get_image_positions(img)
          
    pos = nx.spring_layout(graph,pos=positions, fixed = graph.nodes())
    labels = nx.get_edge_attributes(graph,'weight')
    

    assigned_nodes = list((n for n in graph if graph.node[n]['seed'] != 'none'))
    unassigned_nodes = list((n for n in graph if graph.node[n]['seed'] == 'none'))

    
    seeds = list((n for n in graph if graph.node[n]['type']=='seed'))
    for x in range(len(seeds)):
        #print "Seed {} is {}".format(seeds[x], colors[x])
        nodes = list((n for n in graph if graph.node[n]['seed']==seeds[x]))
        nx.draw_networkx_nodes(graph,pos,
                       nodelist=nodes,
                       node_color=colors[(x % (len(colors) - 1)) + 1],
                       node_size=node_size,
                   alpha=0.8)
        nx.draw_networkx_nodes(graph,pos,
                       nodelist=seeds,
                       node_color=colors[(x % (len(colors) - 1)) + 1],
                       node_size=node_size,
                   alpha=0.8, node_shape= 's')
    
    nx.draw_networkx_nodes(graph,pos,
                       nodelist=unassigned_nodes,
                       node_color=colors[0],
                       node_size=node_size,
                   alpha=0.8)

        
    
    if pixel_values:
        values = nx.get_node_attributes(graph,'value')
        nx.draw_networkx_labels(graph,pos,labels=values)
    else:
        nx.draw_networkx_labels(graph,pos)

    nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels)
    nx.draw_networkx_edges(graph,pos , width=5,edge_color='b')
    
    plt.savefig(figurename, dpi=100)
    
    plt.imshow(img, cmap = 'gray')
    
    plt.savefig(figurename, dpi=100)

In [12]:
def plant_seeds(graph, seeds):
    
    temp_graph = graph.copy()
    
    types_dict = dict()
    assignment_dict = dict()
    assignment_history = dict()

    for x in temp_graph.nodes():
        if x in seeds:
            types_dict[x] = "seed"
            assignment_dict[x] = x
            assignment_history[x] = []
        else:
            types_dict[x] = "node"
            assignment_dict[x] = 'none'
            assignment_history[x] = []
        
            
    nx.set_node_attributes(temp_graph, "type", types_dict)
    nx.set_node_attributes(temp_graph, "seed", assignment_dict)
    nx.set_node_attributes(temp_graph, "assignment history", assignment_history)
    
    return temp_graph

In [13]:
def generate_seeds(graph, n_seeds):
    
    seeds = []
    
    for x in range(n_seeds):
        seeds.append(random.choice(graph.nodes()))
        
    return seeds

In [14]:
def get_graph(window_size, window_tl, image):
    
    fname = "Saved_Graphs/graph_{}_{}_{}_{}.p".format(window_tl[0],window_tl[1],window_size[0],window_size[1])
    
    #if graph already exists, load graph
    if os.path.isfile(fname):
        
        print("Retrieveing graph.")
        graph = nx.read_gpickle(fname)
        
    #else, build the graph    
    else:
        
        print("Graph doesn't exist. Creating graph.")
        #Translate image to 4 connected grid graph
        graph = img_to_graph(image)
        nx.write_gpickle(graph,fname)
        
    return graph

In [15]:
def get_graph_seed_assignments(graph):
    assignments = dict()
    c = count(0)
    for seed in seeds:
        a = next(c)
        nodes = list((n for n in graph if graph.node[n]['seed']==seeds[a]))
        assignments.update(dict.fromkeys(nodes, a))
    return assignments

In [16]:
def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    
    return np.array([(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in colors])

In [17]:
def projections(graph, seed, image):
    '''This will output the relative labelings and save the RGB image to the relative labelings folder'''
    
    positions = get_image_positions(image)
    assignments = get_graph_seed_assignments(graph)
    
    seeds = set(assignments.values())
    n_seeds = len(seeds)
    
    seed_colors = get_spaced_colors(n_seeds)
    
    mask = np.zeros((image.shape[0], image.shape[1], 3))
    
    s = pd.Series(assignments)
    s = s.values.reshape(image.shape)
    
    for x in range(n_seeds):
        a = s == x
        if x == seed:
            mask[a] = [0, 255, 0]
        else:
            mask[a] = [0, 0, 255]
    return mask

In [18]:
def prims_msf(graph):
    iters = []
    G = graph.copy()
    MSF = nx.Graph()
    nodes = G.nodes()
    s = filter(lambda (n, d): d['type'] == 'seed', G.nodes(data=True))
    seeds = []
    iteration = itertools.count(0)
    for x in s:
        seeds.append(x[0])
    push = heappush
    pop = heappop

    while nodes:
        frontier = []
        visited = []
        for u in seeds:
            nodes.remove(u)
            visited.append(u)
            
            # Add seed to MSF
            MSF.add_node(u, attr_dict=G.node[u])
            
            # Push all edges
            for u, v in G.edges(u):
                push(frontier, (G[u][v].get('weight', 1), u, v))
        
        

        while frontier:
            i = next(iteration)
            W, u, v = pop(frontier)   

            if v in visited:
                continue
            
            # Assign the node
            G.node[v]['seed'] = G.node[u]['seed']
            
            # Add node and edge to MSF
            MSF.add_node(v,attr_dict=G.node[v])
            MSF.add_edge(u,v,attr_dict=G.edge[u][v])
            
            G.node[G.node[u]['seed']]['assignment history'] = G.node[G.node[u]['seed']]['assignment history'] + [v]       
            
            visited.append(v)
            nodes.remove(v)
            for v, w in G.edges(v):
                if not w in visited:
                    push(frontier, (G[v][w].get('weight', 1), v, w))
                    
            if i % 10 == 0:
                #view_graph(img, sub_G, figurename="gif/iteration_{}".format(i))
                continue
    return MSF

In [19]:
def field_of_view(master_image, center_pixel, crop_size):
    """Returns a crop of the master image centered on a specified pixel.
    
    Args:
        master_image (numpy.ndarray): The master image to be cropped.
        center_pixel (tuple): The pixel to be at the center of the crop.
        window_size (tuple): The dimensions of the newly cropped image.
        
    Returns:
        A cropped image of the master image with the specified pixel at the center.
        
    Examples:
        fov = field_of_view(I, (0,0), (50,50))"""
    
    center = (center_pixel[0] + crop_size[0], center_pixel[1] + crop_size[1])

    t_l = (center[0] - crop_size[0] // 2, center[1] - crop_size[1] // 2)
    
    return master_image[t_l[0]:t_l[0] + crop_size[0],t_l[1]:t_l[1] + crop_size[1],:]

In [20]:
def assignment_mask(image, graph):
    positions = get_image_positions(image)
    assignments = get_graph_seed_assignments(graph)
    
    seeds = set(assignments.values())
    n_seeds = len(seeds)
    
    seed_colors = get_spaced_colors(n_seeds)
    
    mask = np.zeros((image.shape[0], image.shape[1], 3))
    
    s = pd.Series(assignments)
    s = s.values.reshape(image.shape)
    
    for x in range(n_seeds):
        a = s == x
        mask[a] = seed_colors[x]
    print(seed_colors[0])
    return mask

In [21]:
def max_arc_topographical_distance(path, graph):
    """Returns the max-arc topographical distance of a path in a graph.
    
    Args:
        path (list): A list of tuples defining the path traversed in graph.
        graph (networkx.classes.graph.Graph): The graph containing the path.
        
    Returns:
        Returns the max-arc topographical distance of path."""

    
    G = graph.subgraph(path)
    
    high = -np.infty
    
    for x, y in G.edges_iter():
        weight = G.edge[x][y]['weight']
        if weight > high:
            high = weight
            
    return high

In [22]:
def get_boundary_probabilities(img):
    """Returns boundary probabilites for an image"""
    
    #Make a placeholder for boundary probabilities coming in next iteration of pipeline
    return np.random.rand(img.shape[0],img.shape[1])

In [23]:
def _AsList(x):
    return x if isinstance(x, (list, tuple)) else [x]

In [24]:
def prims_on_demand(G, image_dict):
    """
    Creates a minimum spanning forest from give graph.
    
    Args:
        G (Graph): Graph to use for minimum spanning forest.
        image_dict (Dictionary): The dictionary of nodes used to calculate a weight.
        
    Returns:
        MSF (Graph): Minimum spanning forest.
    """
    
    MSF = nx.Graph()
    nodes = G.nodes()
    
    s = filter(lambda (n, d): d['type'] == 'seed', G.nodes(data=True))
    seeds = []
    
    node_used = dict()
    
    iteration = itertools.count(1)
    
    for x in s:
        seeds.append(x[0])
        
    push = heappush
    pop = heappop
    
    num_edges = G.number_of_edges()

    while nodes:
        
        frontier = []
        visited = []
        
        for u in seeds:
            nodes.remove(u)
            visited.append(u)
            
            # Add seed to MSF
            MSF.add_node(u, attr_dict=G.node[u])
            
            # Push all edges
            for u, v in G.edges(u):
                
                stdout.write("\rCalculating edge {}/{}".format(next(iteration), num_edges))
                stdout.flush()
                
                # Fetch cropped image from dictionary
                fov = image_dict[(v[0], v[1])]
                
                # Calculate newly discovered edge weight
                x = np.expand_dims(fov, axis=0)
                with sess.as_default():
                    weight = sess.run([f_static], feed_dict={image_placeholder: x})[0][0][0]
                    
                G.edge[u][v]['weight'] = weight
                G.edge[u][v]['f_static_image'] = fov
                G.edge[u][v]['f_dynamic_image'] = fov

                push(frontier, (weight, u, v))
        
        

        while frontier:
            W, u, v = pop(frontier)   
            
            if v in visited:
                continue
            
            # assign the node
            G.node[v]['seed'] = G.node[u]['seed']
            
            # add node and edge to MSF
            MSF.add_node(v,attr_dict=G.node[v])
            MSF.add_edge(u,v,attr_dict=G.edge[u][v])

            #p = projections(G, G.node[u]['seed'], image)
            #plt.imshow(p,cmap='jet')
            #savefig('projections_i_{}.png'.format(i))
            G.node[G.node[u]['seed']]['assignment history'] = \
            G.node[G.node[u]['seed']]['assignment history'] + [v]       
            
            visited.append(v)
            nodes.remove(v)
            for v, w in G.edges(v):
                if not w in visited:
                    stdout.write("\rCalculating edge {}/{}".format(next(iteration), num_edges))
                    stdout.flush()
                    
                    # Fetch cropped image from dictionary
                    fov = image_dict[(w[0], w[1])]
                    
                    # Calculate newly discovered edge weight
                    x = np.expand_dims(fov, axis=0)
                    with sess.as_default():
                        weight = sess.run([f_static], feed_dict={image_placeholder: x})[0][0][0]
                    G.edge[v][w]['weight'] = weight
                    G.edge[v][w]['f_static_image'] = fov
                    G.edge[v][w]['f_dynamic_image'] = fov
                
                    push(frontier, (weight, v, w))
                    
            #if i % 10 == 0:
                #view_graph(img, G, figurename="gif/iteration_{}".format(i))
                #continue
    stdout.write("\n")
    
    return MSF

In [25]:
def max_arc_topographical(graph, path):
    
    G = graph.subgraph(path)
    
    high = -np.infty
    edge = None
    
    for x, y in G.edges_iter():
        weight = G.edge[x][y]['weight']
        if weight > high:
            high = weight
            edge = (x,y)
            
    return high, edge

In [26]:
def epoch_data(img_graph):
    """
    Returns arrays containing edge calculation images and edge children count.
    
    Parameters
    ===========
    img_graph: (graph)
    """
    
    
    images = list()
    weights = list()

    for edge in img_graph.edges_iter():
        try:
            images.append(img_graph.edge[edge[0]][edge[1]]['f_static_image'])
            weights.append(img_graph.edge[edge[0]][edge[1]]['error_weights'])
        except KeyError:
            weights.append(0)  

    images = np.array(images)
    weights = np.array(weights)
    
    return images, weights

# Start

# Make Model

In [63]:
class Watershed:
    
    def __init__(self, window_size=(32,32)):
        self.window_size = window_size
    
    def fit(self, images, ground_truth_images):
        """
        Fits the model given an image or set of images.
        
        Args:
            images: A `numpy.ndarray` or list of arrays to be used to fit the model.
            ground_truth_images: A `numpy.ndarray` or list of arrays to be used to fit the model.
        """
        
        self.images = _AsList(images)
        self.ground_truth_images = _AsList(ground_truth_images)
        
        for image, ground_truth_image in zip(self.images, self.ground_truth_images):
            self.image = image
            self.ground_truth_image = ground_truth_image
            self._train_single()
        
    def _train_single(self):
        """
        Trains the model on a single image. Training is composed of two steps. First the image is
        segmented. Second, the model is updated. Before the model is trained, the ground truth
        cut edges are calculated.
        """
        
        self.ground_truth_cuts = self._compute_ground_truth_cuts(self.ground_truth_image, self.seeds)
        
        for i in xrange(10):
            print "\n==========="
            print "Epoch ", i + 1
            print "==========="
            
            print "1. Computing the MSF and constrained MSF"
            self.segment()
            self.training_epoch()
            
            #filename = "image_graph_epoch_{}.jpg".format(i)
            
            #view_graph(img, self.image_graph, figurename=filename)
            #a = 1/0
        
    def _compute_ground_truth_cuts(self, ground_truth_image, seeds):
        """
        Computes the ground truth cuts of the given image.
        
        Args:
            ground_truth_image (ndarray): The image used to create the ground truth cuts.
            seeds (list): A list of seeds to start watershed.
        
        Returns:
            list: A list of ground truth cut edges.
        """
        
        self.ground_truth_graph = img_to_graph(ground_truth_image)
        self.ground_truth_graph = plant_seeds(self.ground_truth_graph, self.seeds)
        self.ground_truth_msf = prims_msf(self.ground_truth_graph)
        
        # Compute the ground truth cut edges
        ground_truth_cuts = []
        [ground_truth_cuts.append(e) if self.ground_truth_msf.node[e[0]]['seed']\
                 is not self.ground_truth_msf.node[e[1]]['seed'] else '' for e in\
                 self.ground_truth_graph.edges_iter()]
        
        return ground_truth_cuts
        

    def segment(self):
        """
        Segments image.
        
        TODO:
            Create and return segmented image.
        """
        
        self._prepare_input_images()
        
        # Translate image to 4 connected grid graph.
        self.image_graph = img_to_graph(self.image)
            
        # Plant seeds.
        self.image_graph = plant_seeds(self.image_graph, self.seeds)
        
        # Compute image MSF.
        self.image_msf =  prims_on_demand(self.image_graph, self.input_images)

        # Compute cut edges
        self.cut_edges = []
        
        [self.cut_edges.append(e) if self.image_msf.node[e[0]]['seed']\
        is not self.image_msf.node[e[1]]['seed'] else '' for e in\
        self.image_graph.edges_iter()]
        
    def training_epoch(self):
        """
        This is the training epoch for each image.  The approach taken here is to compute
        the root error edges, then train the model with the structured data.
        """
        
        
        self.constrained_msf = self._compute_constrained_msf()
        
        # Fetch the training data from the image graph.
        # Reduce amount of data because my computer sucks :(
        images, weights = epoch_data(self.image_graph)
        
        images = images[0:32, :, :, :]
        weights = weights[0:32]
        
        print "2. Identifying root edges and loss."
        self.compute_root_error_edge_children()
        
        with sess.as_default():
            loss_val = sess.run(loss, feed_dict={image_placeholder: images,
                                           root_edges: weights})
            print "Loss = ", loss_val

        print "3. Obtain updated parameters vector and apply gradients."
        with sess.as_default():
            sess.run(tf.global_variables_initializer())
            sess.run(train_step, feed_dict={image_placeholder: images,
                                           root_edges: weights})
            
            
            
        
    
    def _prepare_input_images(self):
        """
        Preprocess images to be used in the prediction of the edges.
        """
        
        self.padded_image = np.pad(self.image,(self.window_size [0],self.window_size [1]),'reflect')
        
        boundary_probabilities = get_boundary_probabilities(self.padded_image)

        #Augment the image with boundary probabilities
        self.augmented_image = np.dstack((self.padded_image, boundary_probabilities))

        # Get input images
        self.input_images = dict()

        for x in range(self.image.shape[0]):
            for y in range(self.image.shape[1]):
                node = (x, y)
                self.input_images[node] = field_of_view(self.augmented_image, node, self.window_size)

        
    def _compute_constrained_msf(self):
        """
        Returns the constained msf.
        
        """
        
        self.img_cuts = self.image_graph.copy()
        self.img_cuts.remove_edges_from(self.ground_truth_cuts)
        return prims_on_demand(self.img_cuts, self.input_images)
        
        
    def compute_root_error_edge_children(self):
        """
        Computes the root error edges used for a single training epoch of the system.
        
        This function will prepare the weight function and the altitude prediction used for the loss.
        The approach taken here is for every node in the graph, check if the node satisfies a failure 
        condition. If so, then add or subtract to the root error edge children.           
        
        By construction of the MSF, the shortest path and the ground truth path are equal
        for all nodes.  Conversely, they differ for incorrect nodes, causing the gound truth
        path distance to exceed the shortest path distance.
        
        TODO:
            Write function to fetch root error false cuts.
        """

        # Create a counter for correct nodes
        correct_nodes = 0
        
        # Create a list of nodes and iterate through them.
        nodes = list((n for n in self.image_msf if self.image_msf.node[n]['type']=='node'))
        
        # Initialize edge error weights
        edge_error_weights = dict()
        
        for node in nodes:
            
            # Get assigned seeds for MSF and constrained MSF.
            assigned_seed = self.image_msf.node[node]['seed']
            ground_truth_seed = self.constrained_msf.node[node]['seed']

            # Get the path from seed to node
            shortest_path = next(nx.all_simple_paths(self.image_msf, assigned_seed, node))
            ground_truth_path = next(nx.all_simple_paths(self.constrained_msf, ground_truth_seed, node))
            
            if shortest_path != ground_truth_path:
                
                root_missing_cut_edge = self._first_erroneous_cut_edge(shortest_path,
                                                                       self.ground_truth_cuts, 
                                                                       self.cut_edges)
                
                root_false_cut_edge = self._compute_root_error_false_cut(self.constrained_msf)
                
                # Apply edge error weights
                try:
                    edge_error_weights[root_missing_cut_edge]
                except KeyError:
                    edge_error_weights[root_missing_cut_edge] = 0
                finally:
                    edge_error_weights[root_missing_cut_edge] = \
                    edge_error_weights[root_missing_cut_edge] + 1
                    
                try:
                    edge_error_weights[root_false_cut_edge]
                except KeyError:
                    edge_error_weights[root_false_cut_edge] = 0
                finally:
                    edge_error_weights[root_false_cut_edge] = \
                    edge_error_weights[root_false_cut_edge] - 1
                    
            else:
                correct_nodes += 1
                continue
            
        print edge_error_weights
        
        print "Correct nodes: {}/{}".format(correct_nodes, self.image_graph.number_of_edges()) 
                
        # Once all nodes have been iterated through, set the edge weights in the graph.
        nx.set_edge_attributes(self.image_graph, 'error_weights', edge_error_weights)
    
    def _compute_root_error_false_cut(self, constrained_msf):
        """
        Computes the root error false cut of a node.
        
        ***Note: This function is not written yet, this will return a random edge of a graph.
        
        TODO:
            Write function.
        """
        
        random_stop = random.randint(1, constrained_msf.number_of_edges())

        for i, edge in enumerate(constrained_msf.edges()):
            if i == random_stop:
                return ((0,0),(0,1))
        else:
            return ((0,0),(0,1))
        
    
    def _first_erroneous_cut_edge(self, shortest_path, ground_truth_cuts, cut_edges):
        """
        Computes the root error missing cut of a shortest path.

        Every incorrect shortest path has at least one erroneous cut edge.  The first such
        edge shall be called the path's root error edge p(w) and is always a missing cut.

        Args:
            shortest_path (list): The list of edges in the shortest path.
            ground_truth_path (list): The list of edges in the ground truth path.

        Returns:
            tuple: The first erroneous cut edge in the shortest path.
        """
        
        for i, node in enumerate(shortest_path):
            try:
                edge = (shortest_path[i], shortest_path[i + 1])
                if edge in ground_truth_cuts or tuple(reversed(edge)) in ground_truth_cuts:
                    if edge not in cut_edges and tuple(reversed(edge)) not in cut_edges:
                        return edge

            except IndexError:
                print "Something went wrong."
                continue
    
    @property
    def ground_truth_cuts(self):
        return self.ground_truth_cuts
    
    
    @property
    def cut_edges(self):
        return self.cut_edges
    
    @property
    def image_msf(self):
        return self.image_msf
    
    
    @property
    def image_graph(self):
        return self.image_graph
    
    
    @property
    def seeds(self):
        """
        Returns the seeds.
        """
        
        return self.seeds
    
    
    @seeds.setter
    def seeds(self, seeds):
        self.seeds = seeds
        
    
    @property
    def window_size(self):
        """
        Returns size of the window used to compute the edge values.
        """
        
        return self.window_size
    
    
    @property    
    def root_edges(self):
        """
        Returns the root error edges list used for the loss of the model.
        
        TODO:
            Actually compute the correct values and don't use the dummy values.
        """
        
        self.root_edges_list = [int(random.randint(-1,1)) for i in xrange(num_of_edges)]
        
        return self.root_edges_list
    
    @property    
    def root_missing_cuts(self):
        """
        Returns the root missing cuts list used for the loss of the model.
        
        TODO:
            Actually compute the correct values and don't use the dummy values.
        """
        
        self.root_edges_list = [int(random.randint(-1,1)) for i in xrange(num_of_edges)]
        
        return self.root_edges_list
    
    @property    
    def root_false_cuts(self):
        """
        Returns the root false cuts list used for the loss of the model.
        
        TODO:
            Actually compute the correct values and don't use the dummy values.
        """
        
        self.root_edges_list = [int(random.randint(-1,1)) for i in xrange(num_of_edges)]
        
        return self.root_edges_list
    
    
    @property
    def edge_values(self):
        """
        Returns the edge values predicted by the model.
        
        TODO:
            Sort the edge values.
        """
        
        self.edge_values = nx.get_edge_attributes(self.image_graph, 'weight').values()
        
        return self.edge_values

In [64]:
# Set image size
image_tl = (50,30)
image_size = (32, 32)
window_size = (15, 15)

#import Images
img = cv2.imread('1O.jpg', 0)
gt = cv2.imread('1G.jpg', 0)

#resize
img = img[image_tl[0]:image_tl[0] + image_size[0],
          image_tl[1]:image_tl[1] + image_size[1]]
gt = gt[image_tl[0]:image_tl[0] + image_size[0],
        image_tl[1]:image_tl[1] + image_size[1]]

#set type
img = img.astype(np.int16)
gt = gt.astype(np.int16)

# Import seeds
seeds = []
f = open("seeds1G.txt", 'r')
for line in f:
    x = int(float(re.split(' ', line)[0]))
    y = int(float(re.split(' ', line)[1]))
    seed = (x, y)

    if x >= image_tl[0] and x <= image_tl[0] + image_size[0]:
        if y >= image_tl[1] and y <= image_tl[1] + image_size[1]:
            x = x - image_tl[0]
            y = y - image_tl[1]
            seed = (x, y)
            seeds.append(seed)

In [65]:
# This placeholder will contain the input images
image_placeholder = tf.placeholder(tf.float32, shape=(None, window_size[0], window_size[0], 2))

# This placeholder will hold the root error edge values.
root_edges = tf.placeholder(tf.float32, shape=(None))

# Create model
m = keras.layers.Conv2D(16, 5, padding = 'same',
                 activation='elu', dilation_rate=1) (image_placeholder)
m = keras.layers.Conv2D(32, 3, padding = 'same',
                 activation='elu', dilation_rate=1) (m)
m = keras.layers.Conv2D(32, 3, padding = 'same',
                 activation='elu', dilation_rate=2) (m)
m = keras.layers.Conv2D(64, 3, padding = 'same',
                 activation='elu', dilation_rate=4) (m)
m = keras.layers.Conv2D(64, 3, padding = 'same',
                 activation='elu', dilation_rate=8) (m)
m = keras.layers.Conv2D(128, 3, padding = 'same',
                 activation='elu', dilation_rate=16) (m)
m = keras.layers.Flatten()(m)
m = keras.layers.Dense(64, activation='relu')(m)
f_static = keras.layers.Dense(1)(m)

#Define loss
loss = tf.reduce_sum(tf.multiply(root_edges, f_static))

# Define optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)

train_step = opt.minimize(loss)

sess = tf.Session() 
sess.run(tf.global_variables_initializer())

In [66]:
w = Watershed(window_size=window_size)

w.seeds = seeds

w.fit(img, gt)


Epoch  1
1. Computing the MSF and constrained MSF
Calculating edge 1984/1984
Calculating edge 1933/1933
2. Identifying root edges and loss.
{((17, 10), (16, 10)): 39, ((0, 0), (0, 1)): -596, ((8, 1), (9, 1)): 482, ((8, 0), (9, 0)): 5, ((18, 18), (17, 18)): 47, ((19, 16), (18, 16)): 19, ((17, 21), (17, 20)): 2, ((17, 10), (17, 11)): 2}
Correct nodes: 426/1984
Loss =  0.0
3. Obtain updated parameters vector and apply gradients.

Epoch  2
1. Computing the MSF and constrained MSF
Calculating edge 1984/1984
Calculating edge 1933/1933
2. Identifying root edges and loss.
{((18, 11), (17, 11)): 601, ((16, 9), (15, 9)): 1, ((0, 0), (0, 1)): -612, ((8, 1), (9, 1)): 4, ((11, 3), (10, 3)): 3, ((16, 9), (16, 10)): 3}
Correct nodes: 410/1984
Loss =  0.0
3. Obtain updated parameters vector and apply gradients.

Epoch  3
1. Computing the MSF and constrained MSF
Calculating edge 1984/1984
Calculating edge 1933/1933
2. Identifying root edges and loss.
{((17, 19), (18, 19)): 1, ((18, 16), (19, 16)): 40,

In [ ]:
def create_batches(data, batch_size):
    """
    Split data into batches of a specific size.  If there is a remainder of data,
    the last batch might be smaller.
    
    Args:
        data (numpy.ndarray):  Data to be split into batches.
        batch_size (int): Size of batches.
    
    Returns:
        batches (list): A list of batches (or numpy arrays), each the specified batch size. 
        
    Examples:
        >>> batches = create_batches(x_train, 32)
    """
    
    batches = list()
    
    while data.shape[0] >= batch_size:
        batches.append(data[0:batch_size,:,:,:])
        data = np.delete(data, range(0,batch_size), 0)

    batches.append(data[0:data.shape[0],:,:,:])
    data = np.delete(data, range(0,data.shape[0]), 0)
    
    return batches

In [ ]:
images, weights = epoch_data(w.image_graph)

In [229]:
w.image_graph.edge[(0,0)][(0,1)]['error_weights']

-209

In [115]:
view_graph(img, w.image_msf, figurename="image_msf.jpg")

In [116]:
view_graph(img, w.constrained_msf, figurename="constrained_msf.jpg")

In [114]:
view_graph(img, w.image_graph, figurename="image_graph.jpg")

In [122]:
def first_erroneous_cut_edge(shortest_path, ground_truth_cuts, cut_edges):
    """
    Computes the root error missing cut of a shortest path.

    Every incorrect shortest path has at least one erroneous cut edge.  The first such
    edge shall be called the path's root error edge p(w) and is always a missing cut.

    Args:
        shortest_path (list): The list of edges in the shortest path.
        ground_truth_path (list): The list of edges in the ground truth path.

    Returns:
        tuple: The first erroneous cut edge in the shortest path.
    """
    
    print shortest_path
    
    for i, node in enumerate(shortest_path):
        try:
            edge = (shortest_path[i], shortest_path[i + 1])
            if edge in ground_truth_cuts or tuple(reversed(edge)) in ground_truth_cuts:
                if edge not in cut_edges and tuple(reversed(edge)) not in cut_edges:
                    print edge, " was a cut edge"
                    return edge

        except IndexError:
            print "Something went wrong."
            continue

In [123]:
nodes = list((n for n in w.image_graph if w.image_graph.node[n]['type']=='node'))

for node in nodes:
    
    m_hat = w.image_msf.node[node]['seed']
    m_star = w.constrained_msf.node[node]['seed']
    
    #Get the path from seed to node
    phi_hat = next(nx.all_simple_paths(w.image_msf, m_hat, node))
    phi_star = next(nx.all_simple_paths(w.constrained_msf, m_star, node))
    
    if phi_hat != phi_star:
        test_shortest_path = phi_hat
        test_ground_truth_path = phi_star
        break    
first_erroneous_cut_edge(phi_hat, w.ground_truth_cuts, w.cut_edges)

[(23, 17), (24, 17), (24, 18), (25, 18), (26, 18), (26, 17), (27, 17), (27, 16), (28, 16), (28, 15), (28, 14), (29, 14), (30, 14), (30, 13), (30, 12), (30, 11), (30, 10), (29, 10), (29, 9), (29, 8), (30, 8), (31, 8), (31, 7), (31, 6)] 

[(8, 1), (9, 1), (9, 2), (10, 2), (11, 2), (11, 3), (11, 4), (10, 4), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (15, 4), (15, 3), (15, 2), (15, 1), (16, 1), (17, 1), (18, 1), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (24, 1), (24, 2), (25, 2), (26, 2), (26, 3), (26, 4), (25, 4), (25, 5), (25, 6), (25, 7), (26, 7), (27, 7), (27, 8), (27, 9), (28, 9), (29, 9), (29, 8), (30, 8), (31, 8), (31, 7), (31, 6)]
((8, 1), (9, 1))  was a cut edge


((8, 1), (9, 1))

In [ ]:
def first_erroneous_cut_edge(shortest_path, ground_truth_path):
    """
    Computes the root error missing cut of a shortest path.

    Every incorrect shortest path has at least one erroneous cut edge.  The first such
    edge shall be called the path's root error edge p(w) and is always a missing cut.

    Args:
        shortest_path (list): The list of edges in the shortest path.
        ground_truth_path (list): The list of edges in the ground truth path.

    Returns:
        tuple: The first erroneous cut edge in the shortest path.
    """
    print shortest_path
    print ground_truth_path

    for i, (shortest_path_node, ground_truth_node) in enumerate(zip(shortest_path, ground_truth_path)):
        #print i, (shortest_path_node, ground_truth_node)
        if shortest_path_node != ground_truth_node:
            print (shortest_path[i - 1], shortest_path[i])
            return (shortest_path[i - 1], shortest_path[i])
    else:
        raise ValueError('No erroneous cut edge.')

In [201]:
weights[edge] = 5

In [202]:
weights[edge]

5

In [204]:
for k, v in weights.iteritems():
    print k, v

((0, 0), (0, 1)) 5


In [207]:


for edge in img_graph.edges_iter():
    try:
        print weights[edge]
    except KeyError:
        #print 0
        continue

5


In [209]:
for edge in img_graph.edges_iter():
    weights[edge] = np.random.randint(-10, high=10)

In [211]:
nx.set_edge_attributes(img_graph, 'error_weights', weights)

In [214]:
x = 10
for i, edge in enumerate(img_graph.edges_iter()):
    print img_graph.edge[edge[0]][edge[1]]
    if i == x:
        break

{'error_weights': -7, 'f_dynamic_image': array([[[  9.80000000e+01,   3.64792847e-01],
        [  7.10000000e+01,   7.25091301e-01],
        [  1.32000000e+02,   4.05334365e-01],
        [  1.34000000e+02,   2.15710940e-01],
        [  1.06000000e+02,   2.77850022e-01],
        [  1.34000000e+02,   2.28169062e-01],
        [  1.32000000e+02,   1.95352215e-01],
        [  7.10000000e+01,   3.07392662e-01],
        [  9.80000000e+01,   9.23107548e-01],
        [  9.90000000e+01,   5.81138928e-01],
        [  8.00000000e+01,   7.96850647e-01],
        [  1.37000000e+02,   5.95013055e-01],
        [  9.30000000e+01,   6.28281372e-01],
        [  7.60000000e+01,   6.11711457e-01],
        [  6.40000000e+01,   4.14662189e-01]],

       [[  9.50000000e+01,   3.77325793e-01],
        [  1.02000000e+02,   1.73868004e-01],
        [  1.23000000e+02,   7.69467802e-02],
        [  1.51000000e+02,   6.95615258e-01],
        [  1.07000000e+02,   7.77217845e-01],
        [  1.51000000e+02,   9.868120

In [29]:
for i, edge in enumerate(w.image_graph.edges_iter()):
    if edge[0][0] == 6:
        print edge

((6, 14), (5, 14))
((6, 14), (7, 14))
((6, 14), (6, 15))
((6, 14), (6, 13))
((6, 1), (6, 2))
((6, 1), (6, 0))
((6, 1), (7, 1))
((6, 18), (6, 19))
((6, 18), (6, 17))
((6, 18), (5, 18))
((6, 18), (7, 18))
((6, 12), (6, 11))
((6, 12), (7, 12))
((6, 12), (5, 12))
((6, 12), (6, 13))
((6, 7), (5, 7))
((6, 7), (6, 6))
((6, 7), (7, 7))
((6, 7), (6, 8))
((6, 16), (6, 15))
((6, 16), (6, 17))
((6, 16), (5, 16))
((6, 11), (7, 11))
((6, 11), (5, 11))
((6, 11), (6, 10))
((6, 2), (6, 3))
((6, 2), (5, 2))
((6, 2), (7, 2))
((6, 5), (6, 4))
((6, 5), (7, 5))
((6, 5), (5, 5))
((6, 5), (6, 6))
((6, 9), (5, 9))
((6, 9), (6, 10))
((6, 9), (6, 8))
((6, 0), (5, 0))
((6, 19), (7, 19))
((6, 10), (7, 10))
((6, 13), (7, 13))
((6, 4), (6, 3))


In [55]:
view_graph(img, w.image_msf)

In [57]:
img_graph = w.image_graph
img_msf = w.image_msf
constrained_msf = w.constrained_msf

In [88]:
def get_test_data():    
    return np.random.rand(5, 15, 15, 2), np.random.randint(-5, high=5, size=(5,1))

In [89]:
test_images, test_weights = get_test_data()

In [98]:
for edge in img_msf.edges_iter():
    img_msf.edge[edge[0]][edge[1]]['children'] = np.random.randint(-10, high=10)

In [119]:
np.array(images)

(1984, 15, 15, 2)

In [142]:
images, weights = epoch_data(w.image_graph)

image_batches = create_batches(images, 32)
weight_batches = create_batches(weights, 32)

# Reduce amount of data because my computer sucks :(
images = images[0:32, :, :, :]
weights = weights[0:32]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(train_step, feed_dict={image_placeholder: images,
                                   root_edges: weights})

In [151]:
create_batches(images, 32)

[array([[[[  9.80000000e+01,   3.64792847e-01],
          [  7.10000000e+01,   7.25091301e-01],
          [  1.32000000e+02,   4.05334365e-01],
          ..., 
          [  9.30000000e+01,   6.28281372e-01],
          [  7.60000000e+01,   6.11711457e-01],
          [  6.40000000e+01,   4.14662189e-01]],
 
         [[  9.50000000e+01,   3.77325793e-01],
          [  1.02000000e+02,   1.73868004e-01],
          [  1.23000000e+02,   7.69467802e-02],
          ..., 
          [  7.80000000e+01,   6.91988722e-02],
          [  7.20000000e+01,   3.46844368e-01],
          [  1.12000000e+02,   7.25885727e-01]],
 
         [[  9.80000000e+01,   5.29993044e-01],
          [  7.10000000e+01,   3.69634304e-01],
          [  1.32000000e+02,   1.26574017e-01],
          ..., 
          [  9.30000000e+01,   5.35719450e-01],
          [  7.60000000e+01,   8.41889793e-02],
          [  6.40000000e+01,   4.65079710e-01]],
 
         ..., 
         [[  1.27000000e+02,   3.92164644e-01],
          [  1.0

In [144]:
            # Get path distance
            t, t_edge = max_arc_topographical(self.image_msf, phi_hat)
            t_star, t_star_edge = max_arc_topographical(self.constrained_msf, phi_star)    

            # The approach I take here is if the node is assigned to the correct seed,
            # then nothing happens.  However, if the node is assigned to the incorrect seed,
            # then two things happen.  First, the image that was used to predict the edge weight is 
            # identified.  This is because when an edge is calculated, it can be calculated from node A
            # to node B, or node B to node A.  Secondly, the image is then added a dictionary  where the 
            # keys are the edges and the values are the images used to calculate the edge.
            
            print self.image_graph.edge[t_edge[0]][t_edge[1]]
        
            if phi_hat == phi_star:
                    continue
            else: 

                # If the node is incorrectly assigned, then 

In [145]:
print images.shape, weights.shape

(32, 15, 15, 2) (32,)


In [62]:
# Where would be the best place to store the children of the root edge.
# I would say that it is best to store that in the netowrkx edge.


def weight_function(MSF, CMSF, node):
    """
    I would like to provide an MSF, CMSF, and a node and get back the row and the row star.
    
    Parameters
    ===========
    MSF: graph
        The minimum spanning forest.
    CMSF: graph
        The Constrained minimum spanning forest.
    node: tuple
        The node to be evaluated.
    """
    
    m_hat = MSF.node[node]['seed']
    m_star = CMSF.node[node]['seed']
    
    #Get the path from seed to node
    phi_hat = next(nx.all_simple_paths(MSF, m_hat, node))
    phi_star = next(nx.all_simple_paths(CMSF, m_star, node))

    #Get path distance
    t = max_arc_topographical_distance(phi_hat, img_msf)
    t_star = max_arc_topographical_distance(phi_star, CMSF)
    
    row = p(MSF, CMSF, node)
    
    row_star = p_star(MSF, CMSF, node)
    
    
    return row, row_star
    

In [58]:
nodes = list((n for n in img_graph if img_graph.node[n]['type']=='node'))
mode_1 = []
mode_2 = []
V_ = []
correct_nodes = []

# {Node, shortest path, ground truth path}
incorrect_paths = {}


c = itertools.count(0)
for node in nodes:
    
    m_hat = img_msf.node[node]['seed']
    m_star = constrained_msf.node[node]['seed']
    
    #Get the path from seed to node
    phi_hat = next(nx.all_simple_paths(img_msf, m_hat, node))
    phi_star = next(nx.all_simple_paths(constrained_msf, m_star, node))

    #Get path distance
    t = max_arc_topographical_distance(phi_hat, img_msf)
    t_star = max_arc_topographical_distance(phi_star, constrained_msf)
    
    print "Node: {} Assigned Seed: {} Path Distance: {}".format(node, m_hat, t)

    if phi_hat == phi_star:
        correct_nodes.append(node)
    else: 
        V_.append(node)
        if m_hat != m_star:
            mode_1.append(node)
            incorrect_paths[node] = [phi_hat, phi_star]
        elif phi_hat != phi_star:
            mode_2.append(node)
            incorrect_paths[node] = [phi_hat, phi_star]
    
    
print len(V_), len(mode_1) + len(mode_2), len(correct_nodes)

Node: (7, 3) Assigned Seed: (8, 1) Path Distance: 0.349301457405
Node: (31, 6) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (5, 31) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (20, 25) Assigned Seed: (23, 17) Path Distance: 0.489943534136
Node: (16, 9) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (6, 28) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (21, 28) Assigned Seed: (23, 17) Path Distance: 0.351304620504
Node: (19, 4) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (17, 20) Assigned Seed: (23, 17) Path Distance: 0.65680372715
Node: (7, 25) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (22, 19) Assigned Seed: (23, 17) Path Distance: 0.521331250668
Node: (20, 7) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (18, 19) Assigned Seed: (23, 17) Path Distance: 0.319327414036
Node: (23, 26) Assigned Seed: (23, 17) Path Distance: 0.485396951437
Node: (21, 6) Assigned Seed: (23, 17) Path Dis

Node: (1, 18) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (25, 13) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (15, 6) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (13, 18) Assigned Seed: (23, 17) Path Distance: 0.554794669151
Node: (2, 25) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (26, 10) Assigned Seed: (23, 17) Path Distance: 0.411369264126
Node: (0, 9) Assigned Seed: (23, 17) Path Distance: 0.307727456093
Node: (3, 4) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (27, 3) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (30, 12) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (4, 7) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (28, 4) Assigned Seed: (23, 17) Path Distance: 0.5775796175
Node: (31, 9) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (5, 6) Assigned Seed: (23, 17) Path Distance: 0.423300743103
Node: (29, 9) Assigned Seed: (23, 17) Path Distanc

Node: (3, 1) Assigned Seed: (23, 17) Path Distance: 0.290696322918
Node: (27, 12) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (30, 11) Assigned Seed: (23, 17) Path Distance: 0.557936906815
Node: (4, 26) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (28, 15) Assigned Seed: (23, 17) Path Distance: 0.327653706074
Node: (31, 2) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (5, 27) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (29, 14) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (16, 13) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (6, 16) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (21, 24) Assigned Seed: (23, 17) Path Distance: 0.532326340675
Node: (19, 8) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (17, 8) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (7, 21) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (22, 23) Assigned Seed: (23, 17) Path

Node: (18, 13) Assigned Seed: (23, 17) Path Distance: 0.537194609642
Node: (23, 16) Assigned Seed: (23, 17) Path Distance: 0.287172853947
Node: (21, 0) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (8, 11) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (22, 15) Assigned Seed: (23, 17) Path Distance: 0.378015100956
Node: (9, 10) Assigned Seed: (23, 17) Path Distance: 0.674091696739
Node: (23, 14) Assigned Seed: (23, 17) Path Distance: 0.303519785404
Node: (10, 1) Assigned Seed: (8, 1) Path Distance: 0.0492432117462
Node: (8, 17) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (11, 12) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (0, 23) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (24, 20) Assigned Seed: (23, 17) Path Distance: 0.522721588612
Node: (14, 11) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (12, 15) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (1, 22) Assigned Seed: (23, 17) Pat

Node: (15, 7) Assigned Seed: (23, 17) Path Distance: 0.319151580334
Node: (13, 19) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (2, 24) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (26, 13) Assigned Seed: (23, 17) Path Distance: 0.426481425762
Node: (0, 8) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (29, 16) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (3, 5) Assigned Seed: (23, 17) Path Distance: 0.360222160816
Node: (27, 0) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (30, 15) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (4, 6) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (28, 11) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (31, 14) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (5, 7) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (29, 10) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (16, 1) Assigned Seed: (23, 17) Path Di

Node: (31, 3) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (5, 28) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (29, 15) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (16, 12) Assigned Seed: (23, 17) Path Distance: 0.375975400209
Node: (6, 19) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (21, 25) Assigned Seed: (23, 17) Path Distance: 0.351304620504
Node: (19, 9) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (17, 9) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (7, 26) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (22, 22) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (20, 2) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (18, 14) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (23, 23) Assigned Seed: (23, 17) Path Distance: 0.293238759041
Node: (21, 3) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (8, 8) Assigned Seed: (23, 17) Path 

Node: (21, 30) Assigned Seed: (23, 17) Path Distance: 0.481672495604
Node: (19, 2) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (17, 22) Assigned Seed: (23, 17) Path Distance: 0.509629189968
Node: (7, 31) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (22, 29) Assigned Seed: (23, 17) Path Distance: 0.3804949224
Node: (20, 5) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (18, 21) Assigned Seed: (23, 17) Path Distance: 0.591242730618
Node: (23, 24) Assigned Seed: (23, 17) Path Distance: 0.306140780449
Node: (8, 3) Assigned Seed: (8, 1) Path Distance: 0.445292085409
Node: (9, 2) Assigned Seed: (8, 1) Path Distance: 0.718392968178
Node: (14, 25) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (10, 25) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (15, 28) Assigned Seed: (23, 17) Path Distance: 0.418789893389
Node: (11, 20) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (0, 31) Assigned Seed: (23, 17) Path Dist

Node: (26, 27) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (0, 26) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (24, 7) Assigned Seed: (23, 17) Path Distance: 0.30269163847
Node: (12, 26) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (27, 18) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (1, 27) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (25, 6) Assigned Seed: (23, 17) Path Distance: 0.448887377977
Node: (13, 27) Assigned Seed: (23, 17) Path Distance: 0.545365810394
Node: (28, 21) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (2, 0) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (26, 5) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (29, 24) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (3, 29) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (27, 8) Assigned Seed: (23, 17) Path Distance: 0.289486408234
Node: (30, 7) Assigned Seed: (23, 17) Path D

In [48]:
mode_1

[]

In [60]:
p = []
p_star1 = []
p_star2 = []
c = itertools.count(0)
for node in incorrect_paths:
    print "Node: ", node, ""
    shortest_path = incorrect_paths[node][0]
    ground_truth_path = incorrect_paths[node][1]
    
    m_hat = img_msf.node[node]['seed']
    m_star = constrained_msf.node[node]['seed']
    
    #Get the path from node to seed
    phi_hat = next(nx.all_simple_paths(img_msf, node, m_hat))
    phi_star = next(nx.all_simple_paths(constrained_msf, node, m_star))
    
    #Calculate Row
    for x in range(len(phi_hat) - 1):
        edge = (phi_hat[x], phi_hat[x + 1])
        if edge in hummus_star:
            if edge not in hummus:
                print "Missing Cut: {}".format(edge)
                p.append(edge)
                break
                #G = img_msf.subgraph(incorrect_paths[node][0])
                #G2 = constrained_msf.subgraph(incorrect_paths[node][1])
                #view_graph(img, G, figurename='paths/a_{}.jpg'.format(next(c)))
                #view_graph(img, G2, figurename='paths/aa_{}.jpg'.format(next(c)))
                
    
    for x in range(len(phi_star) - 1):
        edge = (phi_star[x], phi_star[x + 1])
        if edge in hummus:
            if edge not in hummus_star:
                p_star1.append(edge)
                print "False Cut: {}".format(edge)
                break
                
    for x in range(len(phi_star) - 1):
        phi_star_edge = (phi_star[x], phi_star[x + 1])
        phi_hat_edge = (phi_hat[x], phi_hat[x + 1])
        if phi_star_edge != phi_hat_edge:
            p_star2.append(edge)
            #print phi_star_edge, "\t", phi_hat_edge
            print "False Cut: ", edge, "\n"
            break
p_star = p_star1 + p_star2

Node:  (31, 6) 


NameError: name 'hummus_star' is not defined

In [ ]:
T_star = []
        self.T = field_of_view(self.augmented_image, (0,0), window_size)
        self.T = np.expand_dims(self.T, axis=0)

        loss = 0

        nodes = list((n for n in self.image_msf if self.image_msf.node[n]['type']=='node'))
        
        for node in nodes:
            
            # Get assigned seeds for MSF and constrained MSF.
            m_hat = self.image_msf.node[node]['seed']
            m_star = self.constrained_msf.node[node]['seed']

            # Get the path from seed to node
            phi_hat = next(nx.all_simple_paths(self.image_msf, m_hat, node))
            phi_star = next(nx.all_simple_paths(self.constrained_msf, m_star, node))

            # Get path distance
            t, t_edge = max_arc_topographical(self.image_msf, phi_hat)
            t_star, t_star_edge = max_arc_topographical(self.constrained_msf, phi_star)    

            if phi_hat == phi_star:
                    continue
            else: 

                # Predict the node
                print t_edge
                node = t_edge[0]
                fov = self.input_images[node]
                x = np.expand_dims(fov, axis=0)
                with sess.as_default():
                    weight = sess.run([f_static], feed_dict={image_placeholder: x})[0][0][0]

                if t != weight:
                    node = t_edge[1]
                    fov = self.input_images[node]
                    x = np.expand_dims(fov, axis=0)
                    with sess.as_default():
                        weight = sess.run([f_static], feed_dict={image_placeholder: x})[0][0][0]
                    if t != weight:
                        print "Error"

                print "{} - {} = {}".format(t_star, t, (t_star - t))
                print "{} - {} = {}".format(t_star, weight, (t_star - weight))

                self.T_star.append(t_star)
                T1 = field_of_view(self.augmented_image, node, self.window_size)
                T1 = np.expand_dims(T1, axis=0)

                self.T = np.concatenate((T, T1),axis=0)
                
                loss = loss + (t_star - t)

        self.T = np.delete(T, 0, 0)

In [77]:
# Set image size
window_tl = (50,30)
window_size = (32, 32)
fov_size = (32, 32)

#import Images
img = cv2.imread('1O.jpg', 0)
gt = cv2.imread('1G.jpg', 0)

#resize
img = img[window_tl[0]:window_tl[0] + window_size[0],
          window_tl[1]:window_tl[1] + window_size[1]]
gt = gt[window_tl[0]:window_tl[0] + window_size[0],
        window_tl[1]:window_tl[1] + window_size[1]]

#set type
img = img.astype(np.int16)
gt = gt.astype(np.int16)

#Pad image for field of view
I = np.pad(img,(fov_size[0],fov_size[1]),'reflect')

boundary_probabilities = get_boundary_probabilities(I)

#Augment the image with boundary probabilities
I_a = np.dstack((I, boundary_probabilities))

In [78]:
# Get input images
input_images = dict()

for x in range(window_size[0]):
    for y in range(window_size[1]):
        node = (x, y)
        input_images[node] = field_of_view(I_a, node, fov_size)

In [89]:
I_a.shape

(96, 96, 2)

In [87]:
for k, v in input_images.iteritems():
    print v.shape

(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32, 32, 2)
(32,

In [108]:
#Translate image to 4 connected grid graph
img_graph = img_to_graph(img)
img_graph.name = "image_graph"
ground_truth_graph = img_to_graph(gt)
ground_truth_graph.name = "ground_truth_graph"

In [250]:
#import seeds
seeds = []
f = open("seeds1G.txt", 'r')
for line in f:
    x = int(float(re.split(' ', line)[0]))
    y = int(float(re.split(' ', line)[1]))
    seed = (x, y)
    
    if x >= window_tl[0] and x <= window_tl[0] + window_size[0]:
        if y >= window_tl[1] and y <= window_tl[1] + window_size[1]:
            x = x - window_tl[0]
            y = y - window_tl[1]
            seed = (x, y)
            seeds.append(seed)

In [251]:
#plant seeds
img_graph = plant_seeds(img_graph, seeds)
ground_truth_graph = plant_seeds(ground_truth_graph, seeds)

In [157]:
feed = [int(random.randint(-1,1)) for i in xrange(num_of_edges)]

In [159]:
with tf.Session() as sess:
    
    sh = sess.run(R_e, feed_dict={R_e: feed})
print sh

[ 1  0 -1 ...,  1 -1 -1]


In [136]:
R_e

<tf.Tensor 'Placeholder_14:0' shape=(1984, 1) dtype=int8>

In [39]:
fov = input_images[(0,0)]
x = np.expand_dims(fov, axis=0)
x.shape

(1, 32, 32, 2)

In [129]:
import keras
import tensorflow as tf

# Dummy input
test = np.random.rand(1, 32, 32, 1)

x = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))

# Define model
l1 = keras.layers.Conv2D(16, 5, padding = 'same', activation='elu') (x)
l2 = keras.layers.Flatten()(l1)
l3 = keras.layers.Dense(128, activation='relu') (l2)
predictions = keras.layers.Dense(1) (l3)

grads = tf.gradients(predictions, [l1, l2, l3])

apply_l1_grads = tf.add(l1, grads[0])
apply_l2_grads = tf.add(l2, grads[1])
apply_l3_grads = tf.add(l3, grads[2])

# Define optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    y = sess.run(predictions, feed_dict={x: test})
    
    # Get gradients of y w.r.t model parameters.
    #gradients = sess.run(tf.gradients(y, model_parameters))

In [130]:
y

array([[-0.02749706]], dtype=float32)

In [131]:
iterations = 200

In [132]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for i in xrange(iterations):
        sess.run([apply_l1_grads, apply_l2_grads, apply_l3_grads], feed_dict={x: test})
        y = sess.run(predictions, feed_dict={x: test})
        print y

[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.02710382]]
[[-0.027

In [133]:
y

array([[-0.02710382]], dtype=float32)

In [71]:
with tf.Session() as sess:
     a = sess.run(tf.gradients(preds, l1))
print a

TypeError: Fetch argument None has invalid type <type 'NoneType'>

In [40]:
with sess.as_default():
    weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]

In [41]:
weight

-0.044026881

In [109]:
#Get MSF
print "Calculating Image MSF"
img_msf = prims_on_demand(img_graph)
img_msf.name = "Image_MSF"
print "Gt MSF"
gt_msf = prims_msf(ground_truth_graph)
gt_msf.name = "GT_MSF"

#Create lists of cut edges
hummus_star = []
trash = [hummus_star.append(e) if gt_msf.node[e[0]]['seed'] is not gt_msf.node[e[1]]['seed'] else '' for e in ground_truth_graph.edges_iter()]

hummus = []
trash = [hummus.append(e) if img_msf.node[e[0]]['seed'] is not img_msf.node[e[1]]['seed'] else '' for e in img_graph.edges_iter()]

#Remove cut edges from graph
gt_cuts = gt_msf.copy()
gt_cuts.remove_edges_from(hummus)

img_cuts = img_graph.copy()
img_cuts.remove_edges_from(hummus_star)

print "Calculating Constrained MSF"
constrained_msf = prims_on_demand(img_cuts)
constrained_msf.name = "constrained_MSF"

Calculating Image MSF


TypeError: prims_on_demand() takes exactly 2 arguments (1 given)

In [33]:
view_graph(img, img_msf, figurename='img_msf.jpg')

1
2


In [ ]:
T_star = []
T = field_of_view(I_a, (0,0), fov_size)
T = np.expand_dims(T, axis=0)

msf = img_msf
cons_msf = constrained_msf

loss = 0
i = 0
a = []

nodes = list((n for n in msf if msf.node[n]['type']=='node'))
for node in nodes:

    m_hat = msf.node[node]['seed']
    m_star = cons_msf.node[node]['seed']

    # Get the path from seed to node
    phi_hat = next(nx.all_simple_paths(msf, m_hat, node))
    phi_star = next(nx.all_simple_paths(cons_msf, m_star, node))

    # Get path distance
    t, t_edge = max_arc_topographical(msf, phi_hat)
    t_star, t_star_edge = max_arc_topographical(cons_msf, phi_star)    
    
    if phi_hat == phi_star:
            continue
    else: 
        
        # Predict the node
        print t_edge
        node = t_edge[0]
        fov = input_images[node]
        x = np.expand_dims(fov, axis=0)
        with sess.as_default():
            weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
        
        if t != weight:
            node = t_edge[1]
            fov = input_images[node]
            x = np.expand_dims(fov, axis=0)
            with sess.as_default():
                weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
            if t != weight:
                print "OH SHIT IT'S AN ALIEN!!"
                
        print "{} - {} = {}".format(t_star, t, (t_star - t))
        print "{} - {} = {}".format(t_star, weight, (t_star - weight))
        
        T_star.append(t_star)
        T1 = field_of_view(I_a, node, fov_size)
        T1 = np.expand_dims(T1, axis=0)
        
        T = np.concatenate((T, T1),axis=0)
        
        i = i + 1
        loss = loss + (t_star - t)
        #f_p = img_graph.edge[t_star_edge[0]][t_star_edge[1]]['weight']
        #print t_star, f_p
        #if t_star != f_p:
        #    print "problem"
        #else:
        #    print "no problem"
            
    
    
T = np.delete(T, 0, 0)
print loss

In [ ]:
len(batches)

In [43]:
# Define cost
cost = tf.reduce_sum(preds)

# Define optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(cost)

with sess.as_default():
    sess.run(tf.global_variables_initializer())
    for x in batches:
        opt.run(feed_dict={image_placeholder: x})
        best = sess.run([preds], feed_dict={image_placeholder: x})

NameError: name 'batches' is not defined

In [ ]:
best

In [ ]:
#Get MSF
print "Image MSF"
img_msf = prims_on_demand(img_graph)
img_msf.name = "Image_MSF"
print "Gt MSF"
gt_msf = prims_msf(ground_truth_graph)
gt_msf.name = "GT_MSF"

#Create lists of cut edges
hummus_star = []
trash = [hummus_star.append(e) if gt_msf.node[e[0]]['seed'] is not gt_msf.node[e[1]]['seed'] else '' for e in ground_truth_graph.edges_iter()]

hummus = []
trash = [hummus.append(e) if img_msf.node[e[0]]['seed'] is not img_msf.node[e[1]]['seed'] else '' for e in img_graph.edges_iter()]

#Remove cut edges from graph
gt_cuts = gt_msf.copy()
gt_cuts.remove_edges_from(hummus)

img_cuts = img_graph.copy()
img_cuts.remove_edges_from(hummus_star)

print "Constrained MSF"
constrained_msf = prims_on_demand(img_cuts)
constrained_msf.name = "constrained_MSF"

In [ ]:
view_graph(img, img_msf, figurename='img_msf.jpg')